In [32]:
import pandas as pd
from input_data import input_df
import scipy.stats
import matplotlib.pyplot as plt
from pre4 import data_pre
from m01_lightgbm import train
from save_model2 import save_model_params
pd.set_option('display.max_rows', 150)

In [2]:
df = input_df()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
df = data_pre(df)

In [4]:
df.head()

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,建築年_en
ID,,,,,,,,,,,,,,,,,,,,,
1117225,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,...,準工業地域,60.0,200.0,2018.75,未改装,NaN,7.431364,42.631246,141.603122,6
1114724,北海道,苫小牧市,青葉町,青葉,9.0,３ＬＤＫ,70.0,35.0,ＲＣ,住宅,...,第１種中高層住居専用地域,60.0,200.0,2017.75,未改装,NaN,6.342423,42.631246,141.603122,13
1114753,北海道,苫小牧市,青葉町,青葉,3.0,３ＬＤＫ,70.0,31.0,ＲＣ,住宅,...,第１種中高層住居専用地域,60.0,200.0,2016.50,改装済,NaN,6.778151,42.631246,141.603122,10
1120806,北海道,苫小牧市,表町,苫小牧,8.0,４ＬＤＫ,105.0,18.0,ＲＣ,住宅,...,商業地域,80.0,400.0,2011.99,未改装,NaN,7.230449,42.631246,141.603122,7
1117264,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,90.0,15.0,ＲＣ,住宅,...,準工業地域,60.0,200.0,2015.50,未改装,NaN,7.204120,42.631246,141.603122,7


In [5]:
df["建築年"] = df["建築年"].fillna(0)

In [6]:
year_list_t = set(df["建築年"].tolist())

In [7]:
year_list = []
for y in year_list_t:
    y = str(y).split('.')[0]
    year_list.append(y)

In [8]:
year_dic = {}
for i in range(76):
    if i <= 3:
        year_dic[i] = i
    elif 4 <= i <= 33:
        year_dic[i] = 4 + i//5
    elif 34 <= i <= 96:
        year_dic[i] = 10 + i//10
    
year_dic[76] = 18

In [9]:
df["建築年_en"] = df["建築年"].replace(year_dic).astype(int)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import lightgbm
import optuna.integration.lightgbm as lgb


df_train, df_val = train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

In [11]:
opt_params = {
    "objective":"regression",
    "metric":"mae"
}

import time
start = time.time()
opt=lgb.train(
    opt_params,
    trains,
    valid_sets = valids,
    verbose_eval=200,
    num_boost_round = 1000,
    early_stopping_rounds = 100,
)
end = time.time()
print('elaptedtime:', end-start)

[I 2021-05-30 01:59:23,150] A new study created in memory with name: no-name-a6dc2731-0f78-4211-b641-3c9916ab3d12
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12383
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219638
Training until validation scores don't improve for 100 rounds


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[200]	valid_0's l1: 0.0787728
[400]	valid_0's l1: 0.0772421
[600]	valid_0's l1: 0.0768441
[800]	valid_0's l1: 0.0767189
[1000]	valid_0's l1: 0.0766081
Did not meet early stopping. Best iteration is:
[998]	valid_0's l1: 0.0766072


feature_fraction, val_score: 0.076607:  14%|#4        | 1/7 [00:28<02:48, 28.00s/it]

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009593 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12383
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219638
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's l1: 0.075598
[400]	valid_0's l1: 0.0747456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	valid_0's l1: 0.0746083
[LightGBM] [Warning] No further splits with positive gain, best

min_data_in_leaf, val_score: 0.074518:  80%|########  | 4/5 [02:26<00:36, 36.78s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009653 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12383
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219638
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0755322
[400]	valid_0's l1: 0.0747171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	valid_0's l1: 0.074548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

min_data_in_leaf, val_score: 0.074506: 100%|##########| 5/5 [03:05<00:00, 37.19s/it]

elaptedtime: 2641.486027240753


In [12]:
# lightGBM
import lightgbm as lgb
best_params = opt.params
model = lgb.train(best_params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=200)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12383
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219638
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0755322
[400]	valid_0's l1: 0.0747171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	valid_0's l1: 0.074548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [13]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07450628775667277

In [33]:
save_model_params(model)

In [14]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,21958
最寄駅：名称,20623
市区町村名,10915
取引時点,4838
建築年,3713
最寄駅：距離（分）,2918
面積（㎡）,2596
間取り,2534
都道府県名,2203
都市計画,2144


In [15]:
df_test = pd.read_csv("/tmp/working/dataset/nishika/old_apartment_2020/test.csv", index_col=0)
df_test = data_pre(df_test)
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("./output/09_sub.csv")

In [111]:
df_test

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,緯度,経度,クラスタ平均,建築年_en
ID,,,,,,,,,,,,,,,,,,,,,
1000000,北海道,札幌市中央区,旭ケ丘,円山公園,26.0,３ＬＤＫ,75.0,32.0,ＲＣ,NaN,...,第１種低層住居専用地域,40.0,60.0,2020.50,未改装,NaN,43.046848,141.322114,2.0,10
1000110,北海道,札幌市中央区,大通西,西１８丁目,2.0,１ＬＤＫ,45.0,30.0,ＳＲＣ,住宅,...,商業地域,80.0,400.0,2020.75,未改装,NaN,43.046848,141.322114,2.0,10
1000111,北海道,札幌市中央区,大通西,西１８丁目,2.0,２ＬＤＫ,50.0,32.0,ＳＲＣ,住宅,...,商業地域,80.0,400.0,2020.75,改装済,NaN,43.046848,141.322114,2.0,10
1000112,北海道,札幌市中央区,大通西,西１８丁目,3.0,２ＬＤＫ,80.0,24.0,ＲＣ,NaN,...,商業地域,80.0,400.0,2020.75,改装済,NaN,43.046848,141.322114,2.0,8
1000113,北海道,札幌市中央区,大通西,西１８丁目,3.0,１Ｋ,20.0,40.0,ＲＣ,NaN,...,商業地域,80.0,400.0,2020.75,未改装,NaN,43.046848,141.322114,2.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47006180,沖縄県,沖縄市,大里,NaN,NaN,２ＬＤＫ,35.0,30.0,ＲＣ,NaN,...,第１種低層住居専用地域,50.0,100.0,2020.75,NaN,NaN,26.334353,127.805606,3.0,10
47006652,沖縄県,沖縄市,比屋根,NaN,NaN,２ＬＤＫ,60.0,37.0,ＲＣ,NaN,...,市街化区域及び市街化調整区域外の都市計画区域,70.0,200.0,2020.75,未改装,NaN,26.334353,127.805606,3.0,13
47006790,沖縄県,沖縄市,美里,NaN,NaN,３ＤＫ,55.0,33.0,ＲＣ,NaN,...,商業地域,80.0,400.0,2020.50,未改装,NaN,26.334353,127.805606,3.0,10
